#Learning to detect a square using one convolutional filter
* Create 1000 random images where 500 of them have squares in them. 
* Train one convolutional filter of size `5 x 5` to classfy these two sets of images.
* Investigate what the filter has learnt and why it works.
* Analyze the correctly and incorrectly classified examples.

In [ ]:
import numpy as np
from skimage import draw
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
X = np.random.random((1000, 10, 10, 1))
Y = np.zeros((1000, 1))

for i in range(0, 1000, 2):
    randomx = 2 + int(np.random.random() * 6)
    randomy = 2 + int(np.random.random() * 6)
    randoml = 1
    rr, cc = draw.rectangle_perimeter( start = (randomx, randomy), extent = (randoml, randoml), shape=(10, 10))
    X[i+1, rr, cc, 0] = 1
    Y[i+1] = 1

In [ ]:
# Visualize first 16 inputs and outputs
L = 4
plt.figure(figsize=(20,16))
for p in range(0, L):
    for q in range(0, L):
        plt.subplot(L, L, p * L + q + 1)
        sns.heatmap(X[p * L + q, :, :, 0], cmap='Blues')
        plt.title('Label: ' + str(Y[p * L + q]))
plt.show()

In [ ]:
print(X.shape, Y.shape)

In [ ]:
# Build a CNN model with only one 5x5 filter
from tensorflow.python.keras.layers import Input, Convolution2D, Activation, GlobalMaxPool2D
from tensorflow.python.keras.models import Model
my_input = Input(shape = (10, 10, 1))
my_output = Convolution2D(1, (5, 5))(my_input)
my_output = Activation('sigmoid')(my_output)
# 6x6 = 36, picks up the maximum
my_output = GlobalMaxPool2D()(my_output)
model = Model(my_input, my_output)

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# baseline acc = 50%

In [ ]:
# Pull out the weights of the filter
filter_weights = model.get_weights()[0]
print(filter_weights.shape)
sns.heatmap(filter_weights[:, :, 0, 0], cmap='Reds')
plt.show()

In [ ]:
model.fit(X, Y, epochs = 512, verbose = 1, batch_size=10)

In [ ]:
# Visualize the filter
filter_weights = model.get_weights()[0]
print(filter_weights.shape)
sns.heatmap(filter_weights[:, :, 0, 0], cmap = 'Reds')
plt.show()

np.set_printoptions(formatter = {'float': '{: 0.1f}'.format})
print(filter_weights[:, :, 0, 0])

In [ ]:
# Create another model which can give us the output of the filter
my_input = Input(shape = (10, 10, 1))
my_output = Convolution2D(1, (5, 5))(my_input)
my_output = Activation('sigmoid')(my_output)
model_intermediate = Model(my_input, my_output)

# Load the weights of our model into this model
model.save_weights('a.hdf5')
model_intermediate.load_weights('a.hdf5')

In [ ]:
# Obtain predictions and output of the CNN filter
P = model.predict(X)
O = model_intermediate.predict(X)

In [ ]:
# Check for misclassified examples
for i in range(0, 1000):
    if np.round(P[i]) == Y[i]:
        continue
    plt.figure(figsize=(10, 4))
    plt.suptitle(str(Y[i]) + '    ' + str(P[i]))
    plt.subplot(1, 2, 1)
    sns.heatmap(X[i, :, :, 0], cmap = 'Blues')
    plt.subplot(1, 2, 2)
    sns.heatmap(O[i, :, :, 0], cmap = 'Oranges')
    plt.show()

In [ ]:
# Check for correctly classified examples
for i in range(0, 10):
    if np.round(P[i]) != Y[i]:
        continue
    plt.figure(figsize=(10, 4))
    plt.suptitle(str(Y[i]) + '    ' + str(P[i]))
    plt.subplot(1, 2, 1)
    sns.heatmap(X[i, :, :, 0], cmap = 'Blues')
    plt.subplot(1, 2, 2)
    sns.heatmap(O[i, :, :, 0], cmap = 'Oranges')
    plt.show()